<a href="https://colab.research.google.com/github/G-FI/ML2023/blob/main/ML2023Spring_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 2: Phoneme Classification**


作业说明：
1. concatenate 11个frame形成pohonma，作为input vector，然后分类该vector的lable
2. 一个窄深，一个宽浅两个model，观察training/validation 精度，训练速度，泛化能力
3. 添加dropout观察状况


**说明**
本次作业大部分在本地完成，不太熟悉pytorch需要调试完成。
将数据集下载并解压到当前目录下

Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [ ]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
seed = 1314
same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda


**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [ ]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

# train_split 中每一行代表一个发音，它索引的.pt文件为该发音对应的frames，需要把它们拼接成phoneme作为网络的输入向量
# label使用dict保存, <发音文件名, 其中fram的分类列表>
def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        # 读取label
        for line in open('/'.join([phone_path, f'{mode}_labels.txt'])).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()

        # 使用同一个random_seed，两次shuffle结果相同，取前一部分做train后一部分做validate
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        # 读取了n个frames
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        #通过concatenate将n个frame(39*n)变为n个phoneme(39*k * n)
        feat = concat_feat(feat, concat_nframes)
        # 标签有41类
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        # 将这一个发音
        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


**对于RNN的数据处理方式**

In [ ]:
"""input:
    frames.shape = (n, 39)
    seq_len 多少个frame作为一个phoneme

  output:
    (n, seq_len, 39) n 个frame，每一个对应(seq_len, 39)的序列
"""
def concat_seq(frames, seq_len):
    num, feat_dim = frames.shape[0], frames.shape[1]
    frames = concat_feat(frames, seq_len)
    frames = frames.view(num, seq_len, feat_dim)
    return frames



# train_split 中每一行代表一个发音，它索引的.pt文件为该发音对应的frames，需要把它们拼接成phoneme作为网络的输入向量
# label使用dict保存, <发音文件名, 其中fram的分类列表>
def preprocess_data_rnn(split, feat_dir, phone_path, seq_len, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        # 读取label
        for line in open('/'.join([phone_path, f'{mode}_labels.txt'])).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()

        # 使用同一个random_seed，两次shuffle结果相同，取前一部分做train后一部分做validate
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, seq_len, 39)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        # 读取了n个frames
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)

        # 剪切掉对seq_len求余之后多余的feature
        feat = concat_seq(feat, seq_len)


        # 标签有41类
        if mode == 'train':
          # 对应剪切多余的label
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label


        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]
      y = y.reshape(len(y),1).repeat(1, seq_len)

    # 每一个sequence有seq_len个feature，原本每个feature对应一个label
    # 先择每个sequence对应一个label，但是在运算时RNN输出为(batch_size, seq_len, hidden_dim)
    # 需要将其变为 (batch_size * seq_len, hidden_dim)只有两个维度的tensor才能连接到FC层
    # 通过全连接层之后输出变为此时model的输出就成了(batch_size * seq_len, out_putdim)
    # 而label若不变的话，仍是(batch_size, 1)维度的，因此在batch_size这个维度不匹配
    # 因此认为一个sequence中的每个feature都对应中间feature对应的label



    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X

# Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            data = self.data[idx]
            label = self.label[idx].reshape(-1)
            return data, label
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

In [ ]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.linear = nn.Linear(input_dim, output_dim)
        nn.init.kaiming_normal_(self.linear.weight, a=0, mode='fan_in', nonlinearity='relu')
        self.block = nn.Sequential(
            self.linear,
            nn.BatchNorm1d(output_dim),
            nn.ReLU(),
            nn.Dropout(p=0.25),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

**简单RNN架构**

In [ ]:
class ClassifierRNN(nn.Module):
    def __init__(self, input_dim, output_dim = 41, hidden_layer=1, hidden_dim=256):
        super(ClassifierRNN, self).__init__()

        self.hidden_dim = hidden_dim
        self.hidden_layer = hidden_layer
        self.output_dim = output_dim
        self.bidirectional = False
        self.rnn = nn.RNN(input_dim, hidden_dim, hidden_layer, nonlinearity='tanh', batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    '''
        input:
            x: shape = (batch_size, seq_len, feature_dim)

        output:
            out: prediction one hot
    '''
    def forward(self, x):

        h0 = self.get_h0(x.shape[0], self.hidden_dim, self.hidden_layer)

        output, hn = self.rnn(x, h0)

        output = output.contiguous().view(-1, self.hidden_dim)

        output = self.fc(output)

        return output

    def get_h0(self, batch_size, hidden_dim, hidden_layer):
        D = 1 if self.bidirectional is False else 2

        h0 = torch.zeros((D * hidden_layer, batch_size, hidden_dim), dtype=torch.float32).to(device)
        return h0



# Hyper-parameters

In [ ]:

# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 11   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.75   # the ratio of data used for training, the rest will be used for validation

# training parameters
batch_size = 512        # batch size
num_epoch = 100         # the number of training epoch
learning_rate = 1e-4      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
# 一个MFCCfeature即frame是39个维度，n个freme拼接在一起才是phoneme，对应于中间frame的phoneme的标签
input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
hidden_layers = 6          # the number of hidden layers
hidden_dim = 512           # the hidden dim

In [ ]:
# set_hyeper_param_for_rnn

# data prarameters
train_ratio = 0.75   # the ratio of data used for training, the rest will be used for validation

# training parameters
batch_size = 512        # batch size
num_epoch = 100         # the number of training epoch
learning_rate = 1e-4      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

seq_len = 11
input_dim = 39

hidden_layers = 2
hidden_dim = 128

# Dataloader

**FC 训练数据**

In [ ]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 2571


2571it [00:05, 496.29it/s]


[INFO] train set
torch.Size([1588590, 429])
torch.Size([1588590])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:01, 468.22it/s]


[INFO] val set
torch.Size([528204, 429])
torch.Size([528204])


**RNN训练数据**

In [ ]:
# RNN dataloader
from torch.utils.data import DataLoader
import gc

# preprocess data
train_X, train_y = preprocess_data_rnn(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', seq_len=seq_len, train_ratio=train_ratio, random_seed=seed)
val_X, val_y = preprocess_data_rnn(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', seq_len=seq_len, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 2571


2571it [00:06, 399.89it/s]


[INFO] train set
torch.Size([1588590, 11, 39])
torch.Size([1588590, 11])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:03, 240.50it/s]


[INFO] val set
torch.Size([528204, 11, 39])
torch.Size([528204, 11])


In [ ]:
# 测试label的shape
next(iter(train_loader))[1].shape

torch.Size([512, 11])

# Training

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import math

writer = SummaryWriter('./run/')

for epoch in range(3600):
    y = math.sin(math.radians(epoch))
    writer.add_scalar('val', y, epoch)

**FC 网络Training**

In [ ]:
# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)

In [ ]:
import time
from torch.utils.tensorboard import SummaryWriter

def train_loop():
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    start_time = time.time()

    best_acc = 0.0

    writer = SummaryWriter("./run")


    for epoch in range(num_epoch):
        train_acc = 0.0
        train_loss = 0.0
        val_acc = 0.0
        val_loss = 0.0

        # training
        model.train() # set the model to training mode
        for i, batch in enumerate(tqdm(train_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(features)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
            train_acc += (train_pred.detach() == labels.detach()).sum().item()
            train_loss += loss.item()

        # validation
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)

                loss = criterion(outputs, labels)

                _, val_pred = torch.max(outputs, 1)
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

        writer.add_scalar('acc/train', train_acc/len(train_set))
        writer.add_scalar('acc/validation', val_acc/len(val_set))

        print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

        # if the model improves, save a checkpoint at this epoch
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print(f'saving model with acc {best_acc/len(val_set):.5f}')

    end_time = time.time()
    print(f"total train time: {int(end_time - start_time)}, avg train time per epoch: {int((end_time - start_time)/num_epoch)}")

In [ ]:
# FC 训练100epoch结果
train_loop()

100%|██████████| 1032/1032 [00:08<00:00, 125.74it/s]


[001/100] Train Acc: 0.43947 Loss: 1.95562 | Val Acc: 0.54278 loss: 1.51321
saving model with acc 0.54278


100%|██████████| 1032/1032 [00:08<00:00, 122.16it/s]


[002/100] Train Acc: 0.52236 Loss: 1.58755 | Val Acc: 0.58118 loss: 1.36302
saving model with acc 0.58118


100%|██████████| 1032/1032 [00:07<00:00, 135.04it/s]


...

[098/100] Train Acc: 0.68655 Loss: 0.98713 | Val Acc: 0.70332 loss: 0.93216


100%|██████████| 1032/1032 [00:07<00:00, 140.89it/s]


[099/100] Train Acc: 0.68732 Loss: 0.98724 | Val Acc: 0.70344 loss: 0.93135


100%|██████████| 1032/1032 [00:07<00:00, 144.59it/s]

[100/100] Train Acc: 0.68668 Loss: 0.98603 | Val Acc: 0.70360 loss: 0.93181
saving model with acc 0.70360
total train time: 5345, avg train time per epoch: 53


In [ ]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

**RNN 网络training**

In [ ]:
import time
from torch.utils.tensorboard import SummaryWriter
def train_loop_rnn():
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    start_time = time.time()

    best_acc = 0.0

    writer = SummaryWriter("./run")


    for epoch in range(num_epoch):
        train_acc = 0.0
        train_loss = 0.0
        val_acc = 0.0
        val_loss = 0.0

        # training
        model.train() # set the model to training mode
        for i, batch in enumerate(tqdm(train_loader)):
            features, labels = batch
            features = features.to(device)

            labels = labels.view(-1).to(device)

            optimizer.zero_grad()
            outputs = model(features)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
            train_acc += (train_pred.detach() == labels.detach()).sum().item()
            train_loss += loss.item()

        # validation
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.view(-1).to(device)
                outputs = model(features)

                loss = criterion(outputs, labels)

                _, val_pred = torch.max(outputs, 1)
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

        writer.add_scalar('acc/train', train_acc/len(train_set))
        writer.add_scalar('acc/validation', val_acc/len(val_set))

        print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

        # if the model improves, save a checkpoint at this epoch
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print(f'saving model with acc {best_acc/len(val_set):.5f}')

    end_time = time.time()
    print(f"total train time: {int(end_time - start_time)}, avg train time per epoch: {int((end_time - start_time)/num_epoch)}")


In [ ]:
model = ClassifierRNN(input_dim=input_dim, hidden_dim=hidden_dim, hidden_layer=hidden_layers).to(device)

In [ ]:
train_loop_rnn()

100%|██████████| 1032/1032 [00:10<00:00, 100.65it/s]


[001/100] Train Acc: 4.28235 Loss: 2.26936 | Val Acc: 4.66612 loss: 2.08540
saving model with acc 4.66612


100%|██████████| 1032/1032 [00:10<00:00, 101.84it/s]


[002/100] Train Acc: 4.88844 Loss: 1.99971 | Val Acc: 4.99775 loss: 1.95491
saving model with acc 4.99775


100%|██████████| 1032/1032 [00:10<00:00, 100.73it/s]


...

[098/100] Train Acc: 6.27324 Loss: 1.47757 | Val Acc: 6.11979 loss: 1.53116


100%|██████████| 1032/1032 [00:10<00:00, 96.90it/s] 


[099/100] Train Acc: 6.27372 Loss: 1.47702 | Val Acc: 6.12904 loss: 1.52838


100%|██████████| 1032/1032 [00:10<00:00, 97.78it/s] 

[100/100] Train Acc: 6.27669 Loss: 1.47621 | Val Acc: 6.12590 loss: 1.53029
total train time: 4781, avg train time per epoch: 47


In [ ]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

0

# Visualizaion

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6007 (pid 16324), started 22:42:35 ago. (Use '!kill 16324' to kill it.)

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:01, 462.39it/s]


[INFO] test set
torch.Size([527364, 429])


In [ ]:
# load test data
test_X = preprocess_data_rnn(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', seq_len=seq_len)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:01, 475.28it/s]

[INFO] test set
torch.Size([527364, 11, 39])


Make prediction.

In [ ]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


In [ ]:
# load model
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, features in enumerate(tqdm(test_loader)):

        batch_size = len(features)
        features = features.to(device)

        outputs = model(features) # Model对于每个feature输出 #seq_len条预测(41维度的向量)，但是输出格式为(batch_size*seq_len, #class)
        outputs = outputs.view(batch_size, seq_len, -1) # 变为 (batch, seq_len, #class)

        # sequence 上每个frame的判断
        _, test_pred = torch.max(outputs, 2) #get the index of the class with the highest probability
        test_pred, _ = torch.mode(test_pred, 1) # 对于一个feature已经得到了#seq_len个预测，投票选择最终的预测结果

        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)

100%|██████████| 1031/1031 [00:15<00:00, 65.74it/s] 


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))